In [ ]:
import torch
import torch.linalg as tla
import numpy as np
import matplotlib.pyplot as plt
import numml.sparse as sp
import numml.krylov

In [ ]:
# Use CUDA if it's available

if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

print(f'Using device {device}')

In [ ]:
# Create our favorite poisson operator

N = 32
A = sp.eye(N)*2. - sp.eye(N, k=1) - sp.eye(N, k=-1)
A_c = A.to(device)

In [ ]:
# linear forcing term, gives sinusoidal looking u

b = torch.linspace(-1, 1, N+2)[1:-1]
b_c = b.to(device)

plt.plot(b, label='rhs')
plt.plot(sp.spsolve(A, b), label='true solution')
plt.plot(numml.krylov.conjugate_gradient(A, b, iterations=15)[0], '--', label='CG soln')
plt.legend()

In [ ]:
# Find some preconditioner w/ same sparsity as A
# optimize residual of last iterate wrt entries of M

M = A.copy().to(device)
M.requires_grad = True

optimizer = torch.optim.Adam([M.data], lr=0.01)
epochs = 5_000
lh = torch.zeros(epochs)

for i in range(epochs):
    optimizer.zero_grad()
    
    x_, res_ = numml.krylov.conjugate_gradient(A_c, b_c, iterations=10, M=M)
    loss = res_[-1]
    loss.backward()
    
    optimizer.step()
    lh[i] = loss.item()
    
    if i % 100 == 0 or i == epochs - 1:
        print(i, loss.item())

In [ ]:
plt.semilogy(lh)

In [ ]:
x, res = numml.krylov.conjugate_gradient(A_c, b_c, iterations=N)
x_m, res_m = numml.krylov.conjugate_gradient(A_c, b_c, M=M, iterations=N)

res = torch.stack(res).cpu()
res_m = torch.stack(res_m).detach().cpu()

In [ ]:
plt.semilogy(res, label='Conjugate Gradient')
plt.semilogy(torch.Tensor(res_m), label='Optimized PCG')
plt.legend()
plt.grid()
plt.xlabel('Iteration')
plt.ylabel('Residual')